In [6]:
# import moduels
import requests # for crawling
from bs4 import BeautifulSoup # for crawling
import pandas as pd # for handle DataFrame
import numpy as np # for handle DataFrame
import os # for handle local environment
import re # for handel str()

In [9]:
# aa = pd.read_csv("BeerVer5.csv")
# aa = pd.DataFrame(aa['name'])
# aa.head()
# aa.to_csv("beer_name.csv")

In [ ]:
class BeerPictureCrawling(object):
    beers = list() # 전역변수 beers
    
    def __init__(self): # instance 변수 사용 안함 
        pass
    
    # 1. 다운 받을 맥주 이름 불러오고 list로 변경
    def To_list(self, column_name):
        self.column_name = column_name
        
        # 맥주의 이름만 추출한 csv 파일 불러오기
        Beer_names_df = pd.read_csv("beer_name.csv", encoding='utf-8')
        beer_names = list(Beer_names_df[column_name])
        beer_names.sort() 

        # beers에 담기 
        for beer in beer_names:
            beer = beer + " beer" # beer를 넣어야 검색 효율성이 증대

            # 특수문자를 제거함으로써 검색 효율성 증대 + 오류 방지
            parsed_beer = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', beer)

            # 구글은 + 문자를 인식하므로, 공백을 + 로 대체 
            parsed_beer = parsed_beer.replace(" ","+")
            
            # beers 리스트에 append 
            BeerPictureCrawling.beers.append(parsed_beer)
            
        return BeerPictureCrawling.beers
            
    # 2. 구글 이미지 검색으로 requests 요청
    def BeerCrawling(self):
        count = 0 # 결과 수를 count
        error_count = 0 # 에러 수를 count
        error_list = list() # error가 뜬 맥주를 확인하기 위한 error list()
        
        ##############################################################################
        
        # 폴더 생성 code
        BASE_DIR = "C:\\Users\\User\\Desktop" # 바탕화면
        PICTURE_DIR = os.path.join(BASE_DIR, 'beer_pictures') # 폴더이름: beer_pictures

        # 만약 경로가 없으면, 새로운 폴더를 생성하겠습니다.
        if not os.path.exists(PICTURE_DIR):
            os.makedirs(PICTURE_DIR) 
        
        ##############################################################################
        
        # Crawling codes    
        for b in BeerPictureCrawling.beers:
            try:
                filename= b.replace("+"," ")[:-5] + ".png" # png 형식의 파일 
                print(filename)
                BASE_URL = "https://www.google.com/search"
                params = {
                    'tbm': 'isch',
                    'q': b,
                    'tbs': 'ic:specific,isc:white'
                }

                resp = requests.get(BASE_URL, params=params)
                # print(resp)
                # print(resp.url)
                
                soup = BeautifulSoup(resp.content, 'html.parser')
                # print(soup)

                beer_tag = str(soup.find_all('a')[40]) # 가져올 이미지 링크가 존재하는 tag
                # print(beer_tag)
                
                if "spell=1" in beer_tag: # "이것을 찾으셨나요?, 수정된 검색어에 대한 결과" tag
                    beer_tag = str(soup.find_all('a')[41])
                    if "nfpr=1" in beer_tag: # "다음 검색어로 대신 검색" tag
                        beer_tag = str(soup.find_all('a')[42])
                    else:
                        pass
                else:
                    pass

                beer_picture_url = beer_tag.split(" ")[-2].replace("src=","").replace('"',"")
                # print(beer_picture_url)

                img_resp = requests.get(beer_picture_url)
                # print(resp2)

                beers_picture_path = os.path.join(PICTURE_DIR, filename)
                with open(beers_picture_path, 'wb') as f:
                    f.write(img_resp.content) # 이미지 파일 쓰기

                count += 1

            except:
                error_list.append(b)
                error_count += 1
                pass

        # 문자열 추가 
        print("전체 맥주의 수: ", len(BeerPictureCrawling.beers)) # 전체 맥주 이름 list 
        print("\n수집된 맥주의 수: ", count) # 실제로 수집된 맥주 수 
        print("\n에러가 난 맥주의 수: ", error_count) # 에러가 발생한 맥주 수 
        print("\n0?", len(BeerPictureCrawling.beers) - (count + error_count)) # 0이 나와야 정상 

In [ ]:
Test = BeerPictureCrawling()

In [ ]:
Test.To_list('name')

In [ ]:
Test.BeerCrawling()

### 이후 수정사항
1. error 가 발생한 맥주를 제거하기 위한 새로운 리스트 생성 후, 전체 데이터에서 삭제하는 과정을 거침
 - 전체 데이터에서 삭제하는 코드는 여기에 존재하지 않음

In [4]:
errorlist = ['1809+beer', 'A+Deal+With+The+Devil+beer', 'Aon+Pecan+Mud+Cake++Bourbon+BarrelAged+beer', 'Black+Giraffe+beer', 'Black+Mac+beer', 'Brasserie+Des+Rocs+Triple+Impériale+beer', 'Café+Del+Bastardo+beer', 'Dugges++Stillwater++Speyside+Cacao+beer', 'Elixir+beer', 'Full+Moon+Pale+Ale+beer', 'Hallo+Ich+Bin+Berliner+Weisse++Passion+Fruit+beer', 'Hilma+beer', 'Hitachino+Nest+Real+Ginger+Ale+beer', 'Hypnopompa++Rum+BarrelAged+beer', 'Jack+Hammer+beer', 'Karpackie+Super+beer', 'Kubelbier+beer', 'La+Lupulosa+beer', 'La+Trappe+Dubbel+Koningshoeven++Dominus+beer', 'Mornington+Peninsula+Brown+Ale+beer', 'OHaras+Leann+Folláin+beer', 'Old+World+Russian+Imperial+Stout+beer', 'Paulaner+HefeWeissbier+Naturtrüb+beer', 'Porter+Boom+beer', 'Shipwreck+IPA+beer', 'Sigtuna+Midvinterblot+2012+++beer', 'Tasty+Juice+beer', 'Temptress+Chocolate+Porter+beer', 'Tsarina+Esra+beer', 'Weihenstephaner+Pils+beer', 'West+Coast+Wheat+Wine+beer', 'Youngs+London+Stout+beer', '\x8eŽatec+Dark+Lager+beer']
todellist = list()

for error in errorlist:
    error = error.replace("+"," ")[:-5]
    todellist.append(error)

todellist

['1809',
 'A Deal With The Devil',
 'Aon Pecan Mud Cake  Bourbon BarrelAged',
 'Black Giraffe',
 'Black Mac',
 'Brasserie Des Rocs Triple Impériale',
 'Café Del Bastardo',
 'Dugges  Stillwater  Speyside Cacao',
 'Elixir',
 'Full Moon Pale Ale',
 'Hallo Ich Bin Berliner Weisse  Passion Fruit',
 'Hilma',
 'Hitachino Nest Real Ginger Ale',
 'Hypnopompa  Rum BarrelAged',
 'Jack Hammer',
 'Karpackie Super',
 'Kubelbier',
 'La Lupulosa',
 'La Trappe Dubbel Koningshoeven  Dominus',
 'Mornington Peninsula Brown Ale',
 'OHaras Leann Folláin',
 'Old World Russian Imperial Stout',
 'Paulaner HefeWeissbier Naturtrüb',
 'Porter Boom',
 'Shipwreck IPA',
 'Sigtuna Midvinterblot 2012  ',
 'Tasty Juice',
 'Temptress Chocolate Porter',
 'Tsarina Esra',
 'Weihenstephaner Pils',
 'West Coast Wheat Wine',
 'Youngs London Stout',
 '\x8eŽatec Dark Lager']